# Mobile App for Lottery Addiction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that enable users to answer questions like:

* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. [The data set](https://www.kaggle.com/datascienceai/lottery-dataset) has data for 3,665 drawings, dating from 1982 to 2018.

## Important functions

We'll start by writing two functions that we'll use often:

* A function that calculates factorials; and
* A function that calculates combinations.

In [1]:
def factorial(n):
    result = 1
    
    for i in range(n,0,-1):
        result *= i
        
    return result

In [2]:
def combinations(n,k):
    return factorial(n)/(factorial(k)*factorial(n-k))

## Probability of winning the big prize

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
* The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [5]:
def one_ticket_probability(numbers):
    """ this function takes in a list 
    of six unique numbers and prints the probability
    of winning in a way that's easy to understand"""
    
    # total number of possible outcomes
    n_outcomes = combinations(49,6) # 6 numbers are sampled from 49
    probability = 1/n_outcomes
    percentage = probability *100
    
    print('''You have a {:.7f}% chance to win the big 
          prize with the numbers {}'''.format(percentage, 
                                              numbers))
    

Let's test the function.

In [6]:
test_1 = [3, 23, 12, 74, 11, 5]
one_ticket_probability(test_1)

You have a 0.0000072% chance to win the big 
          prize with the numbers [3, 23, 12, 74, 11, 5]


## Historical data check

For the first version of the app, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The institute wants us to consider the data coming from the national 6/49 lottery game in Canada. The data set contains historical data for 3,665 drawings, dating from 1982 to 2018

In [7]:
import pandas as pd

data = pd.read_csv('649.csv')

In [8]:
data.shape

(3665, 11)

In [9]:
data.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


The columns **NUMBER DRAWN 1**,..., **NUMBER DRAWN 6** represent the winning lottery numbers.

## Compare with historical data

Next, we're going to write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The engineering team told us that we need to be aware of the following details:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as a Python list and serve as an input to our function.
* The engineering team wants us to write a function that prints:
    * the number of times the combination selected occurred in the Canada data set; and
    * the probability of winning the big prize in the next drawing with that combination.

In [10]:
def extract_numbers(row):
    """ this function takes as input a row 
    of the lottery dataframe and returns 
    a set containing all the six winning numbers."""
    
    numbers = row[4:10]
    return set(numbers.values)

In [11]:
winning_numbers = data.apply(extract_numbers, axis=1)

In [12]:
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

In [13]:
def check_historical_occurence(user_n, winning_n):
    user_n_set = set(user_n)
    
    check = user_n_set == winning_n
    n_matches = check.sum()
    
    if n_matches == 0:
        print('''The combination {} has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination {} are 0.0000072%.'''.format(user_n, user_n))
        
    else:
        print('''The number of times combination {} has occured in the past is {}.
Your chances to win the big prize in the next drawing using the combination {} are 0.0000072%.'''.format(user_n, n_matches))


In [14]:
test_2 = [3, 36, 37, 19, 8, 41]
check_historical_occurence(test_2, winning_numbers)

The combination [3, 36, 37, 19, 8, 41] has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination [3, 36, 37, 19, 8, 41] are 0.0000072%.


## Multi-ticket probability

Below, we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

We've talked with the engineering team and they gave us the following information:

* The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
* Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
* The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [15]:
def multi_ticket_probability(n):
    
    n_outcomes = combinations(49,6)
    probability = n/n_outcomes
    percentage = probability*100
    
    if n == 1:
        print('''Your chances to win the big prize with one ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(percentage, int(n_outcomes)))
    
    else:
        combinations_simplified = round(n_outcomes / n)   
        print('''Your chances to win the big prize with {:,} different tickets are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n, percentage,
                                                               combinations_simplified))

In [16]:
test = [1, 10, 100, 10000, 1000000, 6991908, 13983816]
for n in test :
    multi_ticket_probability(n)
    print('___________________________________')

Your chances to win the big prize with one ticket are 0.000007%.
In other words, you have a 1 in 13,983,816 chances to win.
___________________________________
Your chances to win the big prize with 10 different tickets are 0.000072%.
In other words, you have a 1 in 1,398,382 chances to win.
___________________________________
Your chances to win the big prize with 100 different tickets are 0.000715%.
In other words, you have a 1 in 139,838 chances to win.
___________________________________
Your chances to win the big prize with 10,000 different tickets are 0.071511%.
In other words, you have a 1 in 1,398 chances to win.
___________________________________
Your chances to win the big prize with 1,000,000 different tickets are 7.151124%.
In other words, you have a 1 in 14 chances to win.
___________________________________
Your chances to win the big prize with 6,991,908 different tickets are 50.000000%.
In other words, you have a 1 in 2 chances to win.
________________________________

## Probability for less numbers

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

In [19]:
def probability_less_6(integer):
    '''this function takes in an integer between 
    2 and 5 and prints information about the chances 
    of winning depending on the value of that integer.'''
    
    n_combinations_ticket = combinations(6, integer)
    n_combinations_remaining = combinations(43, 6 - integer)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining
    
    n_total_outcomes = combinations(49,6)
    probability = successful_outcomes / n_total_outcomes
    percentage = probability*100
    
    combinations_simplified = round(n_total_outcomes/successful_outcomes) 
    
    print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(integer, percentage,
                                                               int(combinations_simplified)))
    

In [20]:
for test_input in [2, 3, 4, 5]:
    probability_less_6(test_input)
    print('--------------------------') # output delimiter

Your chances of having 2 winning numbers with this ticket are 13.237803%.
In other words, you have a 1 in 8 chances to win.
--------------------------
Your chances of having 3 winning numbers with this ticket are 1.765040%.
In other words, you have a 1 in 57 chances to win.
--------------------------
Your chances of having 4 winning numbers with this ticket are 0.096862%.
In other words, you have a 1 in 1,032 chances to win.
--------------------------
Your chances of having 5 winning numbers with this ticket are 0.001845%.
In other words, you have a 1 in 54,201 chances to win.
--------------------------
